VC2 TP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random
import os
import cv2 as cv

# Load the dataset
data_dir = r"/kaggle/input/natural-images/natural_images"
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,d))]
print(classes)

def dataset(path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize all images to 224x224
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), std=[0.5, 0.5, 0.5])
    ])

    data_images = torchvision.datasets.ImageFolder(
        root=path,
        transform=transform
    )

    train_size = int(0.7*len(data_images))
    test_size = len(data_images) - train_size

    train_data, test_data = torch.utils.data.random_split(data_images,
                                                          [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=32, shuffle=True, num_workers=4)

    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=32, shuffle=False, num_workers=4)

    return train_loader, test_loader

# Get train and test loaders
train_loader, test_loader = dataset(data_dir)

# Get a sample of data from the train_loader
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show the dimensions of the images and labels
print(f"Dimensions of the image batch: {images.shape}")
print(f"Dimensions of the labels: {labels.shape}")

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        # Definition of convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 14 * 14, 512)  # Adjusted for 224x224 input
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Application of convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, device="cpu"):
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

        # Evaluate the model after each epoch
        evaluate_model(model, test_loader, criterion, device)

def evaluate_model(model, test_loader, criterion, device="cpu"):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(test_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Initialize the model, criterion, and optimizer
num_classes = len(classes)
model = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)

['motorbike', 'airplane', 'flower', 'dog', 'fruit', 'car', 'cat', 'person']
Dimensions of the image batch: torch.Size([32, 3, 224, 224])
Dimensions of the labels: torch.Size([32])
Epoch [1/10], Loss: 0.8827
Validation Loss: 0.5281, Accuracy: 79.76%
Epoch [2/10], Loss: 0.4747
Validation Loss: 0.4160, Accuracy: 83.91%
Epoch [3/10], Loss: 0.3529
Validation Loss: 0.3327, Accuracy: 87.97%
Epoch [4/10], Loss: 0.3025
Validation Loss: 0.3101, Accuracy: 88.50%
Epoch [5/10], Loss: 0.2608
Validation Loss: 0.2730, Accuracy: 89.61%
Epoch [6/10], Loss: 0.2284
Validation Loss: 0.3169, Accuracy: 88.07%
Epoch [7/10], Loss: 0.2044
Validation Loss: 0.2827, Accuracy: 89.08%
Epoch [8/10], Loss: 0.1770
Validation Loss: 0.2609, Accuracy: 90.05%
Epoch [9/10], Loss: 0.1587
Validation Loss: 0.2706, Accuracy: 90.14%
Epoch [10/10], Loss: 0.1490
Validation Loss: 0.2908, Accuracy: 89.13%
